In [1]:
import sys
#import time
#sys.path.append("..")

import importlib.resources as resources
import pathlib as pl

import pandas as pd
import numpy as np
from munch import Munch
from wigner_time import connection as con
from wigner_time import timeline as tl
from wigner_time import adwin as adwin
from wigner_time import constructor as construct
import wigner_time

from copy import deepcopy

import ADwin

from pyueye import ueye
from pypyueye import Camera

%matplotlib qt

In [38]:
timeline=tl.stack(
    init(),
    trigger_camera(0,2e-4,context="AI"),
    trigger_camera(3,2e-4,context="AI")
#    tl.wait(10)
)

In [39]:
output=adwin.output(adwin.add(timeline,connections,devices))

In [40]:
m = ADwin.ADwin(1)

In [41]:
m = adwin.initialize_ADwin(m, output)

time_end: 3.0002000000000004s


In [42]:
m.Start_Process(1)

In [12]:
adwin.add(timeline,connections,devices)

,time,variable,value,context,module,channel,unit_range,safety_range,value_digits,cycle
0,0.0000,lockbox_MOT__MHz,0.00,ADwin_LowInit,3.0,8.0,"(-200, 200)",None,32768.0,-2
1,0.0000,coil_compensationX__A,0.25,ADwin_LowInit,4.0,7.0,"(-3, 3)","(-3, 3)",35499.0,-2
2,0.0000,coil_compensationY__A,1.50,ADwin_LowInit,3.0,2.0,"(-3, 3)","(-3, 3)",49152.0,-2
3,0.0000,coil_MOTlowerPlus__A,0.10,ADwin_LowInit,4.0,2.0,"(-5, 5)","(-5, 5)",33423.0,-2
4,0.0000,coil_MOTupperPlus__A,-0.10,ADwin_LowInit,4.0,4.0,"(-5, 5)","(-5, 5)",32113.0,-2
5,0.0000,AOM_MOT,1.00,ADwin_LowInit,1.0,1.0,NaN,NaN,1.0,-2
6,0.0000,AOM_repump,1.00,ADwin_LowInit,1.0,2.0,NaN,NaN,1.0,-2
7,0.0000,AOM_OP_aux,0.00,ADwin_LowInit,NaN,NaN,NaN,NaN,NaN,-2
8,0.0000,AOM_OP,1.00,ADwin_LowInit,1.0,31.0,NaN,NaN,1.0,-2
9,0.0000,shutter_MOT,0.00,ADwin_LowInit,1.0,11.0,NaN,NaN,0.0,-2


### Basic data structures

In [3]:
connections = con.connection(
    ["shutter_MOT", 1, 11],
    ["shutter_repump", 1, 12],
    ["shutter_OP001", 1, 14],
    ["shutter_OP002", 1, 15],
    ["AOM_MOT", 1, 1],
    ["AOM_repump", 1, 2],
#    ["AOM_OP_aux", 1, 30],  # should be set to 0 always
    ["AOM_OP", 1, 31],
    ["coil_compensationX__A", 4, 7],
    ["coil_compensationY__A", 3, 2],
    ["coil_MOTlower__A", 4, 1],
    ["coil_MOTupper__A", 4, 3],
    ["coil_MOTlowerPlus__A", 4, 2],
    ["coil_MOTupperPlus__A", 4, 4],
    ["lockbox_MOT__MHz", 3, 8],
)

devices = pd.DataFrame(
    columns=["variable", "unit_range", "safety_range"],
    data=[
        ["coil_compensationX__A", (-3, 3), (-3, 3)],
        ["coil_compensationY__A", (-3, 3), (-3, 3)],
        ["coil_MOTlower__A", (-5, 5), (-5, 5)],
        ["coil_MOTupper__A", (-5, 5), (-5, 5)],
        ["coil_MOTlowerPlus__A", (-5, 5), (-5, 5)],
        ["coil_MOTupperPlus__A", (-5, 5), (-5, 5)],
        #["lockbox_MOT__V", (-10, 10)],
        ["lockbox_MOT__MHz", (-200, 200)],
    ],
)

# I dislike global variables but, unfortunately, a reference will probably still need to be available in the same namespace as these functions for convenience.
constants = Munch(
    safety_factor=1.1,
#    factor__VpMHz=0.05,
    lag_MOTshutter=2.48e-3,
    Compensation=Munch(
        Z__A=-0.1,
        Y__A=1.5,
        X__A=0.25,
    ),
    OP=Munch(
        lag_AOM_on=15e-6,
        lag_shutter_on=1.5e-3,
        lag_shutter_off=1.5e-3,
        duration_shutter_on=140e-6,
        duration_shutter_off=600e-6,
    ),
    AI=Munch(
        lag_shutter_on=2.2e-3,
        lag_shutter_off=1.9e-3,
    )
)


# TODO: Move device conversions out of here?
# Idea would be to make ...__A, __MHz etc. variables at the 'top' level of timeline conversion and then we could run the conversion functions later.
# class lock_box:
#     def to_V(MHz):
#         return constants.factor__VpMHz * MHz

### Fundamental sample preparation functions

In [4]:
def init(**kwargs) :
    """
    Creates an experimental timeline for the initialization of every device.
    """
    return tl.create(
        lockbox_MOT__MHz=0.0,
        coil_compensationX__A= constants.Compensation.X__A,
        coil_compensationY__A= constants.Compensation.Y__A,
        coil_MOTlowerPlus__A = -constants.Compensation.Z__A,
        coil_MOTupperPlus__A =  constants.Compensation.Z__A,
        AOM_MOT = 1,
        AOM_repump = 1,
#        AOM_OP_aux = 0, # TODO: USB-controlled AOMs should be treated on a higher level
        AOM_OP = 1,
        shutter_MOT = 0,
        shutter_repump = 0,
        shutter_OP001 = 0,
        shutter_OP002 = 1,
        context="ADwin_LowInit",
        **kwargs
    )


def MOT(duration=15, lA=-1., uA=-0.98, **kwargs) :
    return tl.stack(
        tl.set(
#           waitChannel = 0,
            shutter_MOT=1,
            shutter_repump=1,
            coil_MOTlower__A=lA,
            coil_MOTupper__A=uA,
            context='MOT',**kwargs
        ),
        tl.wait(duration)
    )
    
def MOT_detunedGrowth(duration=100e-3, durationRamp=10e-3, toMHz=-5, pt=3, **kwargs) :
    return tl.stack(
        tl.next(
            lockbox_MOT__MHz=toMHz,
            t=durationRamp,
            fargs={"ti": pt},context='MOT',**kwargs
        ),
        tl.wait(duration-durationRamp,#"lockbox_MOT__MHz",
                )
    )

def molasses(duration=5e-3, durationCoilRamp=9e-4, durationLockboxRamp=1e-3, toMHz=-90, coil_pt=3, lockbox_pt=3, **kwargs) :

    return tl.stack(
        tl.next(
            coil_MOTlower__A=0,
            coil_MOTupper__A=0, # TODO: can these be other than 0 (e.g. for more perfect compensaton?)
            t=durationCoilRamp,
            fargs={"ti": coil_pt},
            context="molasses",
            **kwargs
        ),
        tl.next(
            lockbox_MOT__MHz=toMHz,
            t=durationLockboxRamp,
            fargs={"ti": lockbox_pt}
        ),
        tl.set(
            shutter_MOT=[duration - constants.lag_MOTshutter,0],
            AOM_MOT=[duration,0],
            relative=True)
    )


In [5]:
tl.display(
tl.stack(
    init(t=-2),
    MOT(9,-1.0,-0.9,t=0.0),
    MOT_detunedGrowth(1,0.1,-5),
    molasses(1,0.1,0.09,-80)
))

### Adding absorption imaging

In [6]:
connections=pd.concat([connections,con.connection(["shutter_imaging", 1, 13],["AOM_imaging", 1, 5],["trigger_camera", 1, 0])])

In [7]:
def trigger_camera(time, exposure, **kwargs) :
    return tl.set(
        ["trigger_camera",[time,1]],
        ["trigger_camera",[time+exposure,0]],
        **kwargs)

def flash_light(time, exposure, **kwargs) :
    sf = constants.safety_factor
    return tl.stack(
        tl.set(
            ["AOM_imaging",[time,1]],
            ["AOM_imaging",[time+exposure,0]],
            **kwargs),
        tl.set(
            ["shutter_imaging",[time-exposure*(sf-1)-constants.AI.lag_shutter_on,1]],
            ["shutter_imaging",[time+exposure*sf,0]]), # TODO: what to write here exactly?
    )

def expose_camera(time, exposure, **kwargs) :
    sf = constants.safety_factor
    return tl.stack(
        flash_light(time,exposure,**kwargs),
        trigger_camera(time-exposure*(sf-1)/2, exposure*sf)
    )

def take_image_plus_Bg(time, exposure, delayBg, **kwargs) :
    return tl.stack(
        expose_camera(time, exposure, **kwargs), # taking At image
        trigger_camera(time+delayBg, exposure*constants.safety_factor), # taking Bg_At image
    )    

def imaging_absorption(time, exposure, delayBg, delayLi, exposureBlow=None, **kwargs) :
    """
    Creates an experimental timeline for absorption imaging.
    """
    context = "imaging_absorption"
    return tl.stack(
        tl.set(AOM_imaging=[time-constants.safety_factor*exposure,0],context=context,**kwargs), # initializing the AOM
        take_image_plus_Bg(time, exposure, delayBg), # taking At + Bg_At image
        flash_light(time+5*delayBg,exposureBlow) if exposureBlow is not None else None, # blow out the atoms in between
        take_image_plus_Bg(time+delayLi, exposure, delayBg), # taking Li + Bg_Li image
    )

In [14]:
tl.display(
    tl.stack(
        init(t=-0.02,shutter_imaging=0,AOM_imaging=1,trigger_camera=0),
        imaging_absorption(1,0.2,0.5,5)
    )
)

In [58]:
MOT(9,-1.0,-0.9,t=0.0)(init())

,time,variable,value,context
0,0.0,lockbox_MOT__MHz,0.00,ADwin_LowInit
1,0.0,coil_compensationX__A,0.25,ADwin_LowInit
2,0.0,coil_compensationY__A,1.50,ADwin_LowInit
3,0.0,coil_MOTlowerPlus__A,0.10,ADwin_LowInit
4,0.0,coil_MOTupperPlus__A,-0.10,ADwin_LowInit
5,0.0,AOM_MOT,1.00,ADwin_LowInit
6,0.0,AOM_repump,1.00,ADwin_LowInit
7,0.0,AOM_OP_aux,0.00,ADwin_LowInit
8,0.0,AOM_OP,1.00,ADwin_LowInit
9,0.0,shutter_MOT,0.00,ADwin_LowInit


In [54]:
MOT_detunedGrowth(1,0.1,-5)

<function funcy.funcs.compose.<locals>.<lambda>.<locals>.<lambda>(*a, **kw)>

In [56]:
molasses(1,0.1,0.09,-80)

TypeError: next.<locals>.<lambda>() missing 1 required positional argument: 'x'

In [29]:
#preparation_functions.MOT(9,-1.0,-0.9,timeline=init())
tl.display(
    tl.stack(
        init(),
        MOT(9,-1.0,-0.9), # without the t argument, display gets confused
        MOT_detunedGrowth(1,0.1,-5),
#        MOT_detunedGrowth(1,0.1,-10),
        molasses(1,0.1,0.09,-80)
    )
)

In [13]:
tl.stack(
    init(),
    MOT(9,-1.0,-0.9))

,time,variable,value,context
0,0.0,lockbox_MOT__MHz,0.00,ADwin_LowInit
1,0.0,coil_compensationX__A,0.25,ADwin_LowInit
2,0.0,coil_compensationY__A,1.50,ADwin_LowInit
3,0.0,coil_MOTlowerPlus__A,0.10,ADwin_LowInit
4,0.0,coil_MOTupperPlus__A,-0.10,ADwin_LowInit
5,0.0,AOM_MOT,1.00,ADwin_LowInit
6,0.0,AOM_repump,1.00,ADwin_LowInit
7,0.0,AOM_OP_aux,0.00,ADwin_LowInit
8,0.0,AOM_OP,1.00,ADwin_LowInit
9,0.0,shutter_MOT,0.00,ADwin_LowInit


In [13]:
#wigner_time.adwin.output(
adwin.add(init(),connections,devices)#)

,time,variable,value,context,module,channel,unit_range,safety_range,value_digits,cycle
0,0.0,lockbox_MOT__MHz,0.00,ADwin_LowInit,3,8,"(-200, 200)",None,NaN,-2
1,0.0,coil_compensationX__A,0.25,ADwin_LowInit,4,7,"(-3, 3)","(-3, 3)",35499.0,-2
2,0.0,coil_compensationY__A,1.50,ADwin_LowInit,3,2,"(-3, 3)","(-3, 3)",49152.0,-2
3,0.0,coil_MOTlowerPlus__A,0.10,ADwin_LowInit,4,2,"(-5, 5)","(-5, 5)",33423.0,-2
4,0.0,coil_MOTupperPlus__A,-0.10,ADwin_LowInit,4,4,"(-5, 5)","(-5, 5)",32113.0,-2
5,0.0,AOM_MOT,1.00,ADwin_LowInit,1,1,NaN,NaN,1.0,-2
6,0.0,AOM_repump,1.00,ADwin_LowInit,1,2,NaN,NaN,1.0,-2
7,0.0,AOM_OP_aux,0.00,ADwin_LowInit,1,30,NaN,NaN,0.0,-2
8,0.0,AOM_OP,1.00,ADwin_LowInit,1,31,NaN,NaN,1.0,-2
9,0.0,shutter_MOT,0.00,ADwin_LowInit,1,11,NaN,NaN,0.0,-2
